# Combine multiple DEM tiles using `gdalwarp`

In [18]:
import pathlib

# Provide paths to input (raw) data, the location where interim merged layers 
# should be output, and the location for final processed annual rasters to be output
raw_dir = pathlib.Path("/gdata1/projects/coastal/intertidal/Elevation_data/Raw")
interim_dir = pathlib.Path("/gdata1/projects/coastal/intertidal/Elevation_data/Interim")
processed_dir = pathlib.Path("/gdata1/projects/coastal/intertidal/Elevation_data/Processed")

## Data sources

Annotated `gdalwarp` command used below:
```
!gdalwarp input_path output_path \ # input and output files
-b 1 \ # band to extract from the input file (only the first band is needed for bathymetry data)
-t_srs EPSG:3577 \ # target spatial reference system (to reproject to the desired coordinate system)
-tr 10 10 \ # output resolution in target SRS units (to resample data to the desired resolution)
-tap \ # align output and target resolution to the same grid (to ensure alignment to a specific grid)
-dstnodata -9999 \ # value to use for nodata pixels in the output file (to ensure nodata pixels are correct)
-r average \ # resampling method to use (to average the values of multiple input pixels when resampling)
-overwrite \ # overwrite the output file if it already exists (to avoid errors if output file already exists)
-multi \ # enable multithreaded processing (to speed up processing by using multiple CPU cores)
-wm 80% \ # working memory to use for caching data during processing (to improve performance by reducing disk I/O)
-co NUM_THREADS=ALL_CPUS \ # number of threads to use for compression (to speed up compression by using multiple CPU cores)
-of COG \ # output format as COG (to create a Cloud Optimized GeoTIFF)
-co COMPRESS=ZSTD \ # compression method to use (to compress the output file using the ZSTD algorithm)
-co PREDICTOR=YES \ # enable the predictor (to improve compression efficiency)
-co OVERVIEWS=AUTO \ # generate overviews automatically (to create overviews for faster access to lower resolution data)
-co OVERVIEW_COUNT=4 # number of overviews to generate (to specify the number of overviews to create)
```

### Combining WA Bathymetry portal data
Will combine all data matching pattern `WA_Govt/*{year}*.bag` into a single output GeoTIFF (compressed in COG format with overviews).


In [ ]:
for year in range(2015, 2024):
    input_path = raw_dir / f"WA_Govt/*{year}*.bag"
    output_path = interim_dir / f"WA_BathymetryPortal_{year}.tif"    
    
    try:
        print(year)
        !gdalwarp $input_path $output_path -b 1 -t_srs EPSG:3577 -tr 10 10 -tap -dstnodata -9999 -r average -overwrite -multi -wm 80% -co NUM_THREADS=ALL_CPUS -of COG -co COMPRESS=ZSTD -co PREDICTOR=YES -co OVERVIEWS=AUTO -co OVERVIEW_COUNT=4
    except:
        print(f"{year} failed to process")

### Combining Uni. of Western Australia Dampier Archipelago LiDAR

Files missing a nodata attribute; requires additional `-srcnodata` flag (i.e. telling `gdal` what nodata pixels are in the source files), in addition to `-dstnodata` (telling `gdal` what we want nodata pixels to be converted to in our combined output TIF).

Assuming [date is 2016 for now](https://www.mdpi.com/2072-4292/12/18/3033); may require confirmation.

In [ ]:
input_path = raw_dir / "WA_UWA_Dampier_Archipelago/*.tif"
output_path = interim_dir / f"WA_BathymetryPortal_{year}.tif" 

!gdalwarp $input_path $output_path -b 1 -t_srs EPSG:3577 -tr 10 10 -tap -srcnodata -32767 -dstnodata -9999 -r average -overwrite -multi -wm 80% -co NUM_THREADS=ALL_CPUS -of COG -co COMPRESS=ZSTD -co PREDICTOR=YES -co OVERVIEWS=AUTO -co OVERVIEW_COUNT=4

### Combining NT Native Title LiDAR

> Warning: this GeoTIFF was slow to produce due to a large number of input files

In [ ]:
input_path = raw_dir / "NT_Native_Title/Clipped_500m/*.asc"
output_path = interim_dir / f"NT_NativeTitleLidar_2021.tif" 

!gdalwarp $input_path $output_path -b 1 -t_srs EPSG:3577 -tr 10 10 -tap -dstnodata -9999 -r average -overwrite -multi -wm 80% -co NUM_THREADS=ALL_CPUS -of COG -co COMPRESS=ZSTD -co PREDICTOR=YES -co OVERVIEWS=AUTO -co OVERVIEW_COUNT=4

### Combining Airborne Research Australia Gulf of Carpentaria Mangrove Dieback LiDAR

Obtained from [Dropbox links](https://www.dropbox.com/s/njq5xfk7cufuihr/ARA_Lidar_Mangroves2017.pdf?dl=0) provided by [ARA](https://www.airborneresearch.org.au/mangrove-dynamics).

Unable to run GDAL merge on DEA Sandbox due to `.bil' not recognized as a supported file format` error; ran on NCI instead (data stored at `/g/data/r78/Airborne_LIDAR/GulfofCarpentaria/`)

In [ ]:
# !gdalwarp GulfofCarpentaria.vrt ARA_GulfCarpentariaLidarMangroves_2017.tif -t_srs EPSG:3577 -tr 10 10 -tap -dstnodata -9999 -r average -overwrite -multi -wm 80% -co NUM_THREADS=ALL_CPUS -of COG -co COMPRESS=ZSTD -co PREDICTOR=YES -co OVERVIEWS=AUTO -co OVERVIEW_COUNT=4


## Inspect outputs

Inspect combined data using GDAL:

In [14]:
output_path = interim_dir / "NT_NativeTitleLidar_2021.tif"

!gdalinfo $output_path

Driver: GTiff/GeoTIFF
Files: /gdata1/projects/coastal/intertidal/Elevation_data/Interim/NT_NativeTitleLidar_2021.tif
Size is 18010, 11106
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 53S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 53S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",135,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
  

## Combine merged rasters into a single annual raster

In [ ]:
for year in range(2015, 2024):
    # Identify merged rasters for given year
    combined_paths = " ".join(str(file) for file in interim_dir.glob(f"*{year}.tif"))
    output_name = processed_dir / f"{year}_combined.tif"

    if not output_name.exists():
        print(output_name)

        # Combine using gdalwarp
        !gdalwarp $combined_paths $output_name -t_srs EPSG:3577 -tr 10 10 -tap -dstnodata -9999 -r average -overwrite -multi -wm 80% -co NUM_THREADS=ALL_CPUS -of COG -co COMPRESS=ZSTD -co PREDICTOR=YES -co OVERVIEWS=AUTO -co OVERVIEW_COUNT=4

/gdata1/projects/coastal/intertidal/Elevation_data/Processed/2015_combined.tif
Creating output file that is 380332P x 108280L.
0Using internal nodata values (e.g. -9999) for image /gdata1/projects/coastal/intertidal/Elevation_data/Interim/WA_BathymetryPortal_2015.tif.
...10...20...30...40...50...60..Using internal nodata values (e.g. -9999) for image /gdata1/projects/coastal/intertidal/Elevation_data/Interim/QLDTileIndex56_Noosa_2015.tif.
.70...80...90...100 - done.
/gdata1/projects/coastal/intertidal/Elevation_data/Processed/2016_combined.tif
Creating output file that is 383135P x 283372L.
0Using internal nodata values (e.g. -9999) for image /gdata1/projects/coastal/intertidal/Elevation_data/Interim/QLDTileIndex56_Bundaberg_2016.tif.
...10...20...30...40...50...60..Using internal nodata values (e.g. -9999) for image /gdata1/projects/coastal/intertidal/Elevation_data/Interim/NSWTileIndex56_Macksville_2016.tif.
.70...80...90...100 - done.
Using internal nodata values (e.g. -9999) for im

In [40]:
# slope_name = output_name.parent / "2021_slope.tif"
# !gdaldem slope $output_name $slope_name -co NUM_THREADS=ALL_CPUS -of COG -co COMPRESS=ZSTD -co PREDICTOR=YES -co OVERVIEWS=AUTO -co OVERVIEW_COUNT=4